In [4]:
from Process import PlateReader
import pandas as pd

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [75]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\organize\05_07_21_Cd_Ligand_Trial_Sample_Info")[0:4]
sample_info

,Total Sonication Time Min,UID,Labware,Slot,Well,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity,CdAc amount mass g,...,Se amount volume uL,OleicAc amount mass g,OleicAc amount volume uL,Oleylamine amount mass g,Oleylamine amount volume uL,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,Octadecene-stock volume uL
0,0,S2_A1_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A1,0.0625,0.25,0.0,0.00,0.072031,...,20.519751,0.00000,0.000000,0.000000,0.000000,625.0,1250.0,0.000000,0.000000,3125.000000
1,0,S2_A2_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A2,0.0625,0.25,0.0,0.25,0.072031,...,20.519751,0.00000,0.000000,0.334363,411.269988,625.0,1250.0,0.000000,411.269988,2713.730012
2,0,S2_A3_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A3,0.0625,0.25,0.5,0.00,0.072031,...,20.519751,0.70615,793.426966,0.000000,0.000000,625.0,1250.0,793.426966,0.000000,2331.573034
3,0,S2_A4_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A4,0.0625,0.25,0.5,0.25,0.072031,...,20.519751,0.70615,793.426966,0.334363,411.269988,625.0,1250.0,793.426966,411.269988,1920.303046


### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [91]:
path = r"C:\Users\Edwin\Downloads\05_10_21_Cd_Ligand_Trial_0min_rerun.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [92]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
Wavelength,300.000,301.000,302.000,303.000,304.000,305.000,306.000,307.000,308.000,309.000,...,791.000,792.000,793.000,794.000,795.000,796.000,797.000,798.000,799.000,800.000
D1,1.272,1.211,1.157,1.108,1.062,1.024,0.984,0.952,0.923,0.898,...,0.112,0.112,0.112,0.112,0.111,0.111,0.111,0.111,0.111,0.111
D2,0.951,0.881,0.826,0.770,0.727,0.687,0.642,0.609,0.578,0.551,...,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.042
D3,1.305,1.245,1.173,1.116,1.057,1.014,0.971,0.930,0.895,0.868,...,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045
D4,2.447,2.373,2.305,2.235,2.162,2.106,2.034,1.975,1.916,1.862,...,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041


***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [93]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
Wavelength,300.000,301.000,302.000,303.000,304.000,305.000,306.000,307.000,308.000,309.000,...,791.000,792.000,793.000,794.000,795.000,796.000,797.000,798.000,799.000,800.000
D1,1.272,1.211,1.157,1.108,1.062,1.024,0.984,0.952,0.923,0.898,...,0.112,0.112,0.112,0.112,0.111,0.111,0.111,0.111,0.111,0.111
D2,0.951,0.881,0.826,0.770,0.727,0.687,0.642,0.609,0.578,0.551,...,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.042
D3,1.305,1.245,1.173,1.116,1.057,1.014,0.971,0.930,0.895,0.868,...,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045
D4,2.447,2.373,2.305,2.235,2.162,2.106,2.034,1.975,1.916,1.862,...,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [106]:
platereader_df = PlateReader.rehead_wavelengths(merged_df)
T = platereader_df.T
platereader_df = T[0:401].T
platereader_df

,300.0nm,301.0nm,302.0nm,303.0nm,304.0nm,305.0nm,306.0nm,307.0nm,308.0nm,309.0nm,...,691.0nm,692.0nm,693.0nm,694.0nm,695.0nm,696.0nm,697.0nm,698.0nm,699.0nm,700.0nm
D1,1.272,1.211,1.157,1.108,1.062,1.024,0.984,0.952,0.923,0.898,...,0.138,0.137,0.137,0.137,0.136,0.136,0.136,0.135,0.135,0.135
D2,0.951,0.881,0.826,0.770,0.727,0.687,0.642,0.609,0.578,0.551,...,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041
D3,1.305,1.245,1.173,1.116,1.057,1.014,0.971,0.930,0.895,0.868,...,0.047,0.047,0.047,0.047,0.047,0.047,0.047,0.047,0.047,0.047
D4,2.447,2.373,2.305,2.235,2.162,2.106,2.034,1.975,1.916,1.862,...,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041


In [107]:
# blank = platereader_df[-1:]
# blank

In [108]:
blanked = platereader_df-blank.values
blanked

,300.0nm,301.0nm,302.0nm,303.0nm,304.0nm,305.0nm,306.0nm,307.0nm,308.0nm,309.0nm,...,691.0nm,692.0nm,693.0nm,694.0nm,695.0nm,696.0nm,697.0nm,698.0nm,699.0nm,700.0nm
D1,0.963,0.927,0.891,0.861,0.830,0.803,0.772,0.749,0.727,0.707,...,0.089,0.088,0.088,0.088,0.087,0.087,0.087,0.086,0.086,0.086
D2,0.642,0.597,0.560,0.523,0.495,0.466,0.430,0.406,0.382,0.360,...,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
D3,0.996,0.961,0.907,0.869,0.825,0.793,0.759,0.727,0.699,0.677,...,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002
D4,2.138,2.089,2.039,1.988,1.930,1.885,1.822,1.772,1.720,1.671,...,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [109]:
sample_info.reset_index(drop=True, inplace=True)
platereader_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, platereader_df], axis = 1) # this is called combine hotdog. 
combined_df

,Total Sonication Time Min,UID,Labware,Slot,Well,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity,CdAc amount mass g,...,691.0nm,692.0nm,693.0nm,694.0nm,695.0nm,696.0nm,697.0nm,698.0nm,699.0nm,700.0nm
0,0,S2_A1_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A1,0.0625,0.25,0.0,0.00,0.072031,...,0.138,0.137,0.137,0.137,0.136,0.136,0.136,0.135,0.135,0.135
1,0,S2_A2_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A2,0.0625,0.25,0.0,0.25,0.072031,...,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041
2,0,S2_A3_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A3,0.0625,0.25,0.5,0.00,0.072031,...,0.047,0.047,0.047,0.047,0.047,0.047,0.047,0.047,0.047,0.047
3,0,S2_A4_05-10-2021,Fischer 24 Well Plate 7400 ÂµL,2,A4,0.0625,0.25,0.5,0.25,0.072031,...,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041,0.041


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [110]:
temp_hold_path = r"05_07_21_Cd_Ligand_Trial_FL_0min_Rerun"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)

,Total Volume,CdOl 84mM volpct,TOP 1mM volpct,ZnOl 84mM volpct,Time Sonicated,200nm,205nm,210nm,215nm,220nm,...,855nm,860nm,865nm,870nm,875nm,880nm,885nm,890nm,895nm,900nm
0,5.0,75.0,25.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,5.0,75.0,25.0,0.0,0.0,-0.007,-0.015,-0.007,-0.007,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.005,-0.006,-0.005,-0.005,-0.005,-0.005
2,5.0,75.0,24.5,0.5,0.0,0.047,0.042,0.052,0.051,0.045,...,-0.007,-0.007,-0.007,-0.007,-0.006,-0.006,-0.006,-0.005,-0.005,-0.005
3,5.0,75.0,24.5,0.5,0.0,0.036,0.036,0.038,0.042,0.040,...,-0.006,-0.006,-0.006,-0.006,-0.004,-0.004,-0.004,-0.003,-0.003,-0.003
4,5.0,75.0,24.0,1.0,0.0,0.027,0.028,0.032,0.038,0.031,...,-0.004,-0.004,-0.004,-0.004,-0.002,-0.003,-0.002,-0.002,-0.002,-0.001
5,5.0,75.0,24.0,1.0,0.0,0.038,0.044,0.046,0.049,0.048,...,-0.005,-0.005,-0.005,-0.005,-0.003,-0.004,-0.003,-0.003,-0.003,-0.002
6,5.0,75.0,20.0,5.0,0.0,0.042,0.044,0.045,0.050,0.048,...,-0.008,-0.008,-0.008,-0.008,-0.006,-0.007,-0.006,-0.006,-0.006,-0.006
7,5.0,75.0,20.0,5.0,0.0,0.046,0.051,0.055,0.055,0.052,...,-0.007,-0.007,-0.007,-0.007,-0.005,-0.006,-0.005,-0.005,-0.005,-0.004
8,5.0,75.0,25.0,0.0,10.0,0.144,0.134,0.148,0.159,0.152,...,-0.005,-0.005,-0.005,-0.005,-0.004,-0.004,-0.003,-0.003,-0.003,-0.003
9,5.0,75.0,25.0,0.0,10.0,0.111,0.101,0.116,0.121,0.116,...,-0.009,-0.008,-0.009,-0.009,-0.008,-0.009,-0.007,-0.007,-0.007,-0.007
